In [ ]:
# starting gpu
!nvidia-smi

Mon Jan 19 11:20:35 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.57.6
Uninstalling transformers-4.57.6:
  Successfully uninstalled transformers-4.57.6
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
  Using cached transformers-4.57.6-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.57.6-py3-none-any.whl (12.0 MB)
Using cached accelerate-1.12.0-py3-none-any.whl (380 kB)


In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk    # loading dataset after loading it from the disk
import matplotlib.pyplot as plt
import pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# Autotokenizer : Text --> token.
# AutoModelForSeq2SeqLM --> pipeline creation

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from transformers import AutoTokenizer, PegasusForConditionalGeneration, AutoModelForSeq2SeqLM

model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

ARTICLE_TO_SUMMARIZE = (
    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly")

inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, return_tensors="pt")    # converting the words into token


summary_ids = model.generate(inputs["input_ids"])  # generating ids for the tokens
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]   # skipping the special tokens like comma, spaces etc, converting to text


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate exampl

"California's largest electricity provider has said it will cut power to more than 300,000 customers for up to a week."

In [ ]:
summary_ids

tensor([[    0,  1310,   131,   116,  1368,  3769,  2089,   148,   243,   126,
           138,   999,   484,   112,   154,   197, 23963,   527,   118,   164,
           112,   114,   396,   107,     1]])

In [ ]:
inputs

{'input_ids': tensor([[14887,   759,  1005,  3163,   126,  2798,   109, 25690,   116,   115,
          1407,   112, 13378,   118,   281,  7213, 10754,  1514,  1047,   107,
           139,  2560,   117,   112,  1329,   109,   887,   113, 39471,   107,
         16502,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
tokenizer

PegasusTokenizerFast(name_or_path='google/pegasus-xsum', vocab_size=96103, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask_2>', 'additional_special_tokens': ['<mask_1>', '<unk_2>', '<unk_3>', '<unk_4>', '<unk_5>', '<unk_6>', '<unk_7>', '<unk_8>', '<unk_9>', '<unk_10>', '<unk_11>', '<unk_12>', '<unk_13>', '<unk_14>', '<unk_15>', '<unk_16>', '<unk_17>', '<unk_18>', '<unk_19>', '<unk_20>', '<unk_21>', '<unk_22>', '<unk_23>', '<unk_24>', '<unk_25>', '<unk_26>', '<unk_27>', '<unk_28>', '<unk_29>', '<unk_30>', '<unk_31>', '<unk_32>', '<unk_33>', '<unk_34>', '<unk_35>', '<unk_36>', '<unk_37>', '<unk_38>', '<unk_39>', '<unk_40>', '<unk_41>', '<unk_42>', '<unk_43>', '<unk_44>', '<unk_45>', '<unk_46>', '<unk_47>', '<unk_48>', '<unk_49>', '<unk_50>', '<unk_51>', '<unk_52>', '<unk_53>', '<unk_54>', '<unk_55>', '<unk_56>', '<unk_57>', '<unk_58>', '<unk_59>', '<unk

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cuda"

In [ ]:
device

'cuda'

# Finetuning LLM

- Every LLM has two components
  - Tokenizer
  - LLM itself

  Why do they exist?
  - Each LLM has their own tokenizer on which they can easily work with and produce meaningful context, else LLM would be total confused about the tokens embeded. The tokenizer controls the size of tokens, this shall be similar to how the LLMs were trained on.

In [ ]:
model = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model)  # load tokenizer
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)  # load model cnn_model

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Downloading the data

!wget https://github.com/krishnaik06/datasets/raw/refs/heads/main/summarizer-data.zip
!unzip summarizer-data.zip

--2026-01-19 11:25:01--  https://github.com/krishnaik06/datasets/raw/refs/heads/main/summarizer-data.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/krishnaik06/datasets/refs/heads/main/summarizer-data.zip [following]
--2026-01-19 11:25:02--  https://raw.githubusercontent.com/krishnaik06/datasets/refs/heads/main/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip.1’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.08s   

2026-01-19 11:25:02 (91.7 MB/s) - ‘summarizer-data.zip.1’ saved [7

In [ ]:
# to load the .arrow files or json  --> datadict
dataset_samsum = load_from_disk('samsum_dataset')
dataset_samsum

In [ ]:
for split in dataset_samsum:
  print(f"{split}: {len(dataset_samsum[split])}")

In [ ]:
print(dataset_samsum["test"][1])

In [ ]:
print(dataset_samsum["test"][1]['dialogue'])

In [ ]:
print(dataset_samsum["test"][1]['summary'])

In [ ]:
# preparing for fine tuning

## Creating function to add inputs_ids, attention mask and labels --> as they are very much essential for re-training or finetuning

def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation=True)

  return {
      'input_ids': input_encodings['input_ids'],
      'attention_mask': input_encodings['attention_mask'],
      'labels': target_encodings['input_ids']
  }

In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

In [ ]:
dataset_samsum_pt # we can notice new columns added such as input_ids, attention_mask, lablels

In [ ]:
dataset_samsum_pt["train"][1]

In [ ]:
## need data-collator for sequence to sequence model

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
trainer = Trainer(model = model_pegasus,
                  args = trainer_args,
                  tokenizer = tokenizer,
                  data_collator = seq2seq_data_collator,
                  train_dataset = dataset_samsum_pt["test"],      # since the train dataset is large in size
                  eval_dataset = dataset_samsum_pt["validation"]
                  )

In [ ]:
trainer.train()

In [ ]:
# evaluation of finetuned model

def generate_batch_sized_chunks(list_of_elements, batch_size):

  """split the dataset into smaller batches that we can process simultaneously
  Yield successive batch-sized chunks from list_of_elements"""

  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset,
                                metric,
                                model,
                                tokenizer,
                                column_text = "article",
                                column_summary = "highlights"):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm(
      zip(article_batches, target_batches),
      total = len(article_batches)):

      inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
      summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                 attention_mask=inputs["attention_mask"].to(device),
                                 length_penalty=0.8,
                                 num_beams=8,
                                 max_length=128)
      ''' Parameter for length penalty ensures that the model does not generate sequences that are too long. '''

      decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]

      decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
      metric.add_batch(predictions=decoded_summaries, references=target_batch)

  score = metric.compute()
  return score


In [ ]:
!pip install evaluate

In [ ]:
import evaluate

rouge_metric = evaluate.load('rouge')
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size =2, column_text = 'dialogue', column_summary='summary'
)

rouge_dict = {rn: score[rn] for rn in rouge_names}


import pandas as pd
pd.DataFrame(rouge_dict, index = [f'pegasus'])

# rouge score closer to 1 is a better metric.

## Interpreting rouge scores

- score ~1 : Strong overlap between the generated summary and reference summary --> highly desirables
- 0.5 < Score < 0.7 : Indicate moderate overlap --> high possibility of missing some information
- score < 0.5: poor match between the generated summary and reference summary.

In [ ]:
# save model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
## save tokenizer
tokenizer.save_pretrained("tokenizer")

In [ ]:
# loading save tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [ ]:
gen_kwargs = {"lengthy_penalty": 0.8, "num_beams": 8, "max_length": 128}

sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model", tokenizer=tokenizer)


##
print('--'*50)
print("Dialogue:")
print(sample_text)


print('--'*50)
print("\n Reference Summary:")
print(reference)

print('--'*50)
print("\n Model Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])